In [ ]:
#@markdown ## Setup dependencies
%%bash

if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi

if [ ! -d alphafold ]; then
  git clone https://github.com/deepmind/alphafold.git
  ! pip -q install ml-collections dm-haiku biopython==1.81
fi

if [ ! -d output ]; then
  mkdir output
fi

In [ ]:
#@markdown ## Define functions to run AlphaFold

import sys
import os
import argparse
import hashlib
import jax
import jax.numpy as jnp
import numpy as np
import re
import subprocess
from glob import glob

sys.path.append('alphafold')

from alphafold.model import model, config, data
from alphafold.data import parsers, pipeline
from alphafold.common import protein

"""
Create an AlphaFold model runner
name -- The name of the model to get the parameters from. Options: model_[1-5]
"""

def make_model_runner(model_num=3, recycles=1, deterministic=True):
  model_name = 'model_%d_ptm' % model_num
  cfg = config.model_config(model_name)

  cfg.data.common.num_recycle = recycles
  cfg.model.num_recycle = recycles
  cfg.data.eval.num_ensemble = 1
  if deterministic:
    cfg.data.eval.masked_msa_replace_fraction = 0.0
    cfg.model.global_config.deterministic = True
  params = data.get_model_haiku_params(model_name, '.')

  return model.RunModel(cfg, params)

def make_processed_feature_dict(runner, a3m_file, name="test", seed=0):
  feature_dict = {}

  # assumes sequence is first entry in msa
  with open(a3m_file,'r') as msa_fil:
    sequence = msa_fil.read().splitlines()[1].strip()

  feature_dict.update(pipeline.make_sequence_features(sequence, name, len(sequence)))

  with open(a3m_file,'r') as msa_fil:
    msa = pipeline.parsers.parse_a3m(msa_fil.read())

  feature_dict.update(pipeline.make_msa_features([msa]))
  processed_feature_dict = runner.process_features(feature_dict, random_seed=seed)

  return processed_feature_dict

"""
Package AlphaFold's output into an easy-to-use dictionary
prediction_result - output from running AlphaFold on an input dictionary
processed_feature_dict -- The dictionary passed to AlphaFold as input. Returned by `make_processed_feature_dict`.
"""
def parse_results(prediction_result, processed_feature_dict):
  b_factors = prediction_result['plddt'][:,None] * prediction_result['structure_module']['final_atom_mask']
  dist_bins = jax.numpy.append(0,prediction_result["distogram"]["bin_edges"])
  dist_mtx = dist_bins[prediction_result["distogram"]["logits"].argmax(-1)]
  contact_mtx = jax.nn.softmax(prediction_result["distogram"]["logits"])[:,:,dist_bins < 8].sum(-1)

  out = {"unrelaxed_protein": protein.from_prediction(processed_feature_dict, prediction_result, b_factors=b_factors),
        "plddt": prediction_result['plddt'],
        "pLDDT": prediction_result['plddt'].mean(),
        "dists": dist_mtx,
        "adj": contact_mtx}

  out.update({"pae": prediction_result['predicted_aligned_error'],
              "pTMscore": prediction_result['ptm']})
  return out

def write_results(result, pdb_out_path):
  plddt = float(result['pLDDT'])
  ptm = float(result["pTMscore"])
  print('plddt: %.3f' % plddt)
  print('ptm  : %.3f' % ptm)

  pdb_lines = protein.to_pdb(result["unrelaxed_protein"])
  with open(pdb_out_path, 'w') as f:
    f.write(pdb_lines)

/usr/local/lib/python3.10/dist-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(


In [ ]:
!pip install mdtraj
import mdtraj as md

gspdb = md.load_pdb('closeCA.pdb')
espdb = md.load_pdb('openCA.pdb')
def calpdb(pdb):
  pdb = md.load_pdb(pdb)
  atoms = pdb.top.select('name CA')
  pdb = pdb.atom_slice(atoms)
  pdb.superpose(gspdb, 0)
  d1 = md.rmsd(pdb, gspdb)
  pdb.superpose(espdb, 0)
  d2 = md.rmsd(pdb, espdb)
  return(d1,d2)


In [ ]:
n_recycles = 3
model_number = 3
seed=0
name='ordered'


runner = make_model_runner(model_num=model_number, recycles=n_recycles)

subsampled_msas = glob('subsampled_MSAs/*.a3m')
subsampled_msas = sorted(subsampled_msas)
print(len(subsampled_msas))
count=0

for fil in subsampled_msas:
  print(fil)
  if os.path.exists('./drive/MyDrive/AdK_seq/erase_es_results/'+ os.path.basename(fil).replace('.a3m','.pdb'))== False:
    features = make_processed_feature_dict(runner, fil, seed=seed)
    result = parse_results(runner.predict(features, random_seed=seed), features)
    pae = np.array(result['pae'])
    np.save('output/pae_' + os.path.basename(fil).replace('.a3m','_0.npy'), pae)
    write_results(result, 'output/' + os.path.basename(fil).replace('.a3m','.pdb'))
    print(calpdb('output/' + os.path.basename(fil).replace('.a3m','.pdb')))
    count += 1
  if count % 10 == 0:
    os.system('scp ./output/* ./drive/MyDrive/AdK_seq/erase_es_results')
    print('finished count:'+str(count))
    #os.system('rm ./output/*')



os.system('scp ./output/* ./drive/MyDrive/AdK_seq/erase_es_results')
print('finished count:'+str(count))